# K-SVD denoising implementation

References:

[1] Nel215. 2017, ksvd, GitHub repository. https://github.com/nel215/ksvd

In [79]:
import cv2
import numpy as np
import scipy as sp
from sklearn.linear_model import orthogonal_mp_gram
from sklearn import linear_model

In [80]:
class KSVD(object):

    """
    Adapted from https://github.com/nel215/ksvd
    """

    def __init__(self, n_components, max_iter=30, tol=5000,
                 n_nonzero_coefs=None):


        self.dictionary = None
        self.sparsecode = None
        self.max_iter = max_iter
        self.tol = tol
        self.n_components = n_components
        self.n_nonzero_coefs = n_nonzero_coefs

    def _initialize(self, y):
  
        u, s, v = np.linalg.svd(y)
        self.dictionary = u[:, :self.n_components]
        # print(self.dictionary.shape)

    def _update_dict(self, y, d, x):

        for i in range(self.n_components):
            index = np.nonzero(x[i, :])[0]
            if len(index) == 0:
                continue

            d[:, i] = 0
            r = (y - np.dot(d, x))[:, index]
            u, s, v = np.linalg.svd(r, full_matrices=False)
            d[:, i] = u[:, 0].T
            x[i, index] = s[0] * v[0, :]
        return d, x

    def fit(self, y):

        self._initialize(y)
        for i in range(self.max_iter):
            x = linear_model.orthogonal_mp(self.dictionary, y, n_nonzero_coefs=self.n_nonzero_coefs)
            e = np.linalg.norm(y - np.dot(self.dictionary, x))
            if e < self.tol:
                break
            self._update_dict(y, self.dictionary, x)

        self.sparsecode = linear_model.orthogonal_mp(self.dictionary, y, n_nonzero_coefs=self.n_nonzero_coefs)
        return self.dictionary, self.sparsecode


In [81]:
import math

def psnr_calculate(img1, img2):
   mse = np.mean( (img1/255. - img2/255.) ** 2 )
   if mse < 1.0e-10:
      return 100
   PIXEL_MAX = 1
   return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))


In [85]:
ksvd = KSVD(128)

img = ['Image No']
psnr = ['PSNR']

for i in range(600):

  # im_gt = cv2.imread("/content/drive/MyDrive/675/Project/datasets/outputs/Original/content/output/im_gt_{}.png".format(i),0).astype(float)
  im_noisy = cv2.imread("/content/drive/MyDrive/675/Project/datasets/outputs/Original/content/output/im_noisy_{}.png".format(i),0).astype(float)
  # result, code = ksvd.fit(im_noisy)
  # im_pred = result.dot(code)
  # im_pred = np.clip(im_pred, 0, 255)

  cv2.imwrite("/content/drive/MyDrive/675/Project/datasets/outputs/grey_noisy/output{}.png".format(i), im_noisy)

  img.append(i)
  psnr.append("%.2f" % psnr_calculate(im_gt,im_pred))

In [39]:
count = np.array(img)
pred_result = np.array(psnr)
result = np.vstack((count,pred_result))
np.savetxt("ksvd.csv", np.transpose(result), delimiter=',',fmt='%s')

#2. PCA denoising implementation


In [75]:
from sklearn.decomposition import PCA
import numpy as np
import scipy.io as sci
import cv2
from pathlib import Path

In [76]:
traning = []

for i in range(600):

  im_noisy = cv2.imread("/content/drive/MyDrive/675/Project/datasets/outputs/Original/content/output/im_noisy_{}.png".format(i),0).astype(float)
  traning.append(im_noisy)

In [77]:
traning_set = np.array(traning)
traning_set = traning_set.reshape(600,65536)

pca = PCA(0.9).fit(traning_set)
components= pca.transform(traning_set)
filtered = pca.inverse_transform(components)
filtered = filtered.reshape(600,256,256)

In [78]:
img = ['Image No']
psnr = ['PSNR']

for i in range(600):
  im_gt = cv2.imread("/content/drive/MyDrive/675/Project/datasets/outputs/Original/content/output/im_gt_{}.png".format(i),0).astype(float)
  im_denoise = filtered[i]

  img.append(i)
  psnr.append("%.2f" % psnr_calculate(im_gt,im_denoise))

count = np.array(img)
pred_result = np.array(psnr)
result = np.vstack((count,pred_result))
np.savetxt("pca.csv", np.transpose(result), delimiter=',',fmt='%s')